In [2]:
import os

path = '/hy-tmp/OpenIBL-master/'
os.chdir(path)
print(os.getcwd())

/hy-tmp/OpenIBL-master


In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.distributed as dist
from ibl import datasets
from ibl.evaluators import Evaluator
from ibl.utils.data.sampler import DistributedSliceSampler
from ibl.utils.data.preprocessor import Preprocessor
from ibl.utils.dist_utils import init_dist, synchronize
from ibl import models
import os
import h5py

In [5]:
def vgg16_netvlad(pretrained=False):
    base_model = models.create('vgg16', pretrained=False)
    pool_layer = models.create('netvlad', dim=base_model.feature_dim)
    model = models.create('embednetpca', base_model, pool_layer)
    if pretrained:
        model.load_state_dict(torch.load('/hy-tmp/vgg16_netvlad.pth', map_location=torch.device('cpu')))
#     base_model = models.create('vgg16', train_layers="conv5", matconvnet='logs/vd16_offtheshelf_conv5_3_max.pth')
#     if True:
#         pool_layer = models.create('netvlad', dim=base_model.feature_dim)
#         # vgg16_pitts_64_desc_cen_mat.hdf5
#         initcache = '/home/private/hyk/OpenIBL/logs/vgg16_pitts_64_desc_cen.hdf5'
#         if (dist.get_rank()==0):
#             print ('Loading centroids from {}'.format(initcache))
#         with h5py.File(initcache, mode='r') as h5:
#             pool_layer.clsts = h5.get("centroids")[...]
#             pool_layer.traindescs = h5.get("descriptors")[...]
#             pool_layer._init_params()

#         model = models.create('embednet', base_model, pool_layer)
#     else:
#         model = base_model

    # if (args.syncbn):
    #     # not work for VGG16
    #     convert_sync_bn(model)
    return model

In [6]:
model = vgg16_netvlad(pretrained=True)

In [7]:
model.eval()
input_x = torch.randn(1, 3, 640, 480)
torch.onnx.export(model, input_x, 'netvlad_dynamic_input.onnx', input_names=['input'], output_names=['output'], verbose='True', dynamic_axes={'input':{0:'batch_size', 2: 'in_height', 3:'in_width'}, 'output':{0:'batch_size', 2: 'out_height', 3:'out_width'}})


graph(%input : Float(*, 3, *, *, strides=[921600, 307200, 480, 1], requires_grad=0, device=cpu),
      %base_model.base.0.weight : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %base_model.base.0.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %base_model.base.2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %base_model.base.2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %base_model.base.5.weight : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %base_model.base.5.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %base_model.base.7.weight : Float(128, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %base_model.base.7.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %base_model.base.10.weight : Float(256, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %base_mod

In [9]:
import onnx
import onnxruntime as ort
import numpy as np

# onnx_model = onnx.load('test.onnx')
# # onnx.checker.check_model(model)
 
session = ort.InferenceSession('netvlad_dynamic_input.onnx')

x=np.random.randn(1,3,640,480).astype(np.float32)  # 注意输入type一定要np.float32!!!!!
# x= torch.randn(batch_size,chancel,h,w)
 
 
onnx_result = session.run(None,{ 'input' : x })
torch_result = model(torch.tensor(x))
np.testing.assert_allclose(torch_result[0].detach().numpy(),onnx_result[0][0],rtol=0.0001)


AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0

Mismatched elements: 116 / 4096 (2.83%)
Max absolute difference: 3.5390258e-07
Max relative difference: 0.0406579
 x: array([ 1.007271e-02, -1.705696e-03, -6.977879e-05, ...,  1.808140e-02,
        3.110385e-02, -3.695749e-02], dtype=float32)
 y: array([ 1.007270e-02, -1.705689e-03, -6.977396e-05, ...,  1.808158e-02,
        3.110395e-02, -3.695760e-02], dtype=float32)

In [10]:
x_1=np.random.randn(1,3,480,640).astype(np.float32)  # 注意输入type一定要np.float32!!!!!
# x= torch.randn(batch_size,chancel,h,w)
 
 
onnx_result = session.run(None,{ 'input' : x_1 })
torch_result = model(torch.tensor(x_1))
np.testing.assert_allclose(torch_result[0].detach().numpy(),onnx_result[0][0],rtol=0.0001)

AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0

Mismatched elements: 142 / 4096 (3.47%)
Max absolute difference: 4.2002648e-07
Max relative difference: 0.01584982
 x: array([ 0.00946 , -0.000971, -0.000524, ...,  0.01391 ,  0.014587,
       -0.034475], dtype=float32)
 y: array([ 0.00946 , -0.000971, -0.000524, ...,  0.013911,  0.014586,
       -0.034475], dtype=float32)